# Coding Applications in Medicine: Data Science - Data Handling Extra

In [1]:
import sqlite3, csv

### Data cleaning/handling outside of Pandas library

In all the other notebooks within this module, we are directly modifying the data after directly reading the data file into Pandas dataframe. There may be instances where you might not be using the Pandas library for the data analysis. In those situations, the data cleaning and processing steps will need to be done via scripts.

The general approach would be to:
1. Read the file and store the corresponding data in data structures
2. Modify the data with user-defined functions
3. Write the data into a new file

References:
- Python3 I/O tutorial: https://docs.python.org/3/tutorial/inputoutput.html
- Python3 CSV I/O tutorial: https://docs.python.org/3/library/csv.html

The following code blocks will mimic the same code we have written in Practice 1 notebook to modify the bmiCategories table.

In [2]:
# User-defined function to modify the bmi value. 
# Category: String with the bmi categorization data.
# Index: 0 for Min and -1 for Max.
def modifyBmi(category, index):
    # Split the bmi category text by ' ' and take the element at the index position of the resulting list.
    bmi = category.split()[index]
    # Check to see if the extracted text is a float (replace only one instance of '.' and check if it is a digit).
    if category.split()[index].replace('.', '', 1).isdigit():
        return float(bmi)
    # Replacement for non-numerical min bmi.
    if index == 0:
        return 0.0
    # Replacement for non-numerical max bmi.
    return 100.0

# Open the file with bmi categories data in read-only format.
with open('data/bmiCategories.csv', 'r') as f:
    # Create a dictionary based on data in the file.
    dr = csv.DictReader(f)
    # Create a list of tuples that contains data obtained from the dictionary.
    # Here we are also introducing the max and min bmi using our previously defined function.
    rowList = [(row["BMI"], row["Weight Status"], modifyBmi(row["BMI"], 0), 
                modifyBmi(row["BMI"], -1)) for row in dr]
    # List of field names used for the header line of the csv we will write later.
    fieldnameList = dr.fieldnames + ["Min BMI", "Max BMI"]

# Open the file with bmi categories data in write format.
# This will create an empty new file.
with open('data/modifiedBmiCategories.csv', 'w', newline='') as f:
    # Writes in csv format (comma as delimiter).
    writer = csv.writer(f, delimiter=',')
    # Writes field names as the first row.
    writer.writerow(fieldnameList)
    # Writes the list of data in row format (new line for each row).
    for row in rowList:
        writer.writerow(row)
        

### Working with databases

In all the other notebooks within this module, we are obtaining the data directly from a file. There may be instances where the data is stored in a database, and you need to export part of the data from the database. One of the most popular languages to handle databases is SQL. In general, if you also need to manipulate the queried data, it is better to do so with scripts after querying and exporting rather than through SQL/within database queries.

References:
- SQLite Documentation: https://www.sqlite.org/docs.html
- Python3 SQLite Tutorial: https://docs.python.org/3/library/sqlite3.html

The following code blocks will mimic the same code we have written we have written in Practice 1 notebook after modifying the bmiCategories table.

In [3]:
# Establishes a connection to the local database file.
# Normally, you would establish a connection to a remote database.
con = sqlite3.connect("data/insurance.db")
cur = con.cursor()

# Checks to see if "insurance" table already exists in the database. 
# If it already exists, then we will clear the data from the table (mainly used for reset convenience).
if cur.execute("SELECT COUNT(*) FROM sqlite_master " +
               "WHERE type = 'table' AND name = 'insurance'").fetchone()[0] > 0:
    cur.execute("DELETE FROM insurance")
    con.commit()

    # This will remove the table instead.
    ###cur.execute("DROP TABLE insurance")
    ###con.commit()

# Creates the "insurance" table (if it does not exist) based on the provided schema.
cur.execute("CREATE TABLE IF NOT EXISTS " + 
                "insurance (age NUMBER(3), sex VARCHAR2(10), bmi NUMBER(6, 3), children NUMBER(2), " + 
                           "smoker VARCHAR2(3), region VARCHAR2(9), charges NUMBER(11, 5));")

# Open the file with the insurance data in a read-only format.
with open("data/insurance.csv", 'r') as f:
    # Create a dictionary based on data in the file.
    dr = csv.DictReader(f)
    # Create a list of tuples that contains data obtained from the dictionary.
    rowList = [(row["age"], row["sex"], row["bmi"], row["children"],
                row["smoker"], row["region"], row["charges"]) for row in dr]

# Bulk insert operation into the "insurance" table.
cur.executemany("INSERT INTO insurance (age, sex, bmi, children, smoker, region, charges) " +
                    "VALUES (?, ?, ?, ?, ?, ?, ?);", rowList)
con.commit()

# Checks to see if "bmicategories" table already exists in the database. 
# If it already exists, then we will clear the data from the table (mainly used for reset convenience).
if cur.execute("SELECT COUNT(*) FROM sqlite_master " + 
               "WHERE type = 'table' AND name = 'bmicategories'").fetchone()[0] > 0:
    cur.execute("DELETE FROM bmicategories")
    con.commit()

    # This will remove the table instead.
    #cur.execute("DROP TABLE bmicategories")
    #con.commit()

# Creates the "insurance" table (if it does not exist) based on the provided schema.
cur.execute("CREATE TABLE IF NOT EXISTS " +
                "bmicategories (BMI VARCHAR2(15), 'Weight Status' VARCHAR2(15), "
                               "'Min BMI' NUMBER(6, 3), 'Max BMI' NUMBER(4, 1));")

# Open the file with the insurance data in a read-only format.
with open("data/modifiedBmiCategories.csv", 'r') as f:
    # Create a dictionary based on data in the file.
    dr = csv.DictReader(f)
    # Create a list of tuples that contains data obtained from the dictionary.
    rowList = [(row["BMI"], row["Weight Status"], row["Min BMI"], row["Max BMI"]) for row in dr]

# Bulk insert operation into the "bmicategories" table.
cur.executemany("INSERT INTO bmicategories (BMI, 'Weight Status', 'Min BMI', 'Max BMI') " +
                    "VALUES (?, ?, ?, ?);", rowList)
con.commit()

In [4]:
# Query statement to obtain data. Generally SQL query statement are readable.
query = ("SELECT bmicategories.'Weight Status', insurance.region, AVG(insurance.charges) " + 
         "FROM insurance FULL OUTER JOIN bmicategories " +
         "WHERE insurance.bmi >= bmicategories.'Min BMI' " +
         "AND insurance.bmi < bmicategories.'Max BMI' " +
         "GROUP BY insurance.region, bmicategories.'Weight Status' " +
         "HAVING insurance.region = 'northeast' " +
         "AND bmicategories.'Weight Status' = 'Overweight';")

# Execute the query statement and fetch the next row (in this case, the first and only row).
res = cur.execute(query).fetchone()

# Select only the relevant data.
resultFromSQL = (res[0].lower(), 
                 res[1], 
                 str(round(res[2], 2)))

print("The average insurance charge for an {0} person living in the {1} region is ${2}."
      .format(resultFromSQL[0], resultFromSQL[1], resultFromSQL[2]))

The average insurance charge for an overweight person living in the northeast region is $10810.64.


In [5]:
# Close the database connection.
con.close()

**Source:**


Module adapted from Kaggle: https://www.kaggle.com/code/mariapushkareva/medical-insurance-cost-with-linear-regression/notebook

Dataset source: https://github.com/stedy/Machine-Learning-with-R-datasets